<a href="https://colab.research.google.com/github/premkrishn/bert-hands-on-nlp/blob/main/SWPv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import os
import re
from urllib.parse import urljoin

def download_website(url, folder):
    # Create folder if it doesn't exist
    if not os.path.exists(folder):
        os.makedirs(folder)

    # Set to keep track of visited URLs
    visited_urls = set()

    # Download the main page and follow internal links recursively
    download_page(url, folder, visited_urls)

def download_page(url, folder, visited_urls):
    try:
        # Check if the URL has already been visited
        if url in visited_urls:
            return
        visited_urls.add(url)

        # Get the webpage content
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # Save page
            save_page(response.content, folder, get_filename(url))

            # Find all links on the page
            links = soup.find_all('a', href=True)
            for link in links:
                href = urljoin(url, link['href'])
                # Check if the link is internal
                if href.startswith(url):
                    # Download the linked page
                    download_page(href, folder, visited_urls)
    except Exception as e:
        print(f"Error downloading {url}: {str(e)}")

def save_page(content, folder, filename):
    with open(os.path.join(folder, filename), 'wb') as f:
        f.write(content)

def get_filename(url):
    # Extract filename from URL
    filename = re.sub(r'^https?://', '', url)
    filename = re.sub(r'[^a-zA-Z0-9\-._]', '_', filename)
    if filename.endswith('/'):
        filename += "index"
    filename += ".html"
    return filename

if __name__ == "__main__":
    # URL of the website
    url = "https://am.jpmorgan.com/us/en/asset-management/liq/"
    # Folder to save web pages
    folder = "jpmorgan_website"

    download_website(url, folder)
    print("Website downloaded successfully!")


In [8]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin
import urllib

# Function to download webpage content
def download_page(url, folder):
    # Make a GET request to the URL
    response = requests.get(url)
    # If the request is successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extract text content
        text_content = soup.get_text()
        # Create a folder for text files if it doesn't exist
        text_folder = os.path.join(folder, "text")
        if not os.path.exists(text_folder):
            os.makedirs(text_folder)
        # Get the page title to use as the filename
        title = soup.title.text.strip() if soup.title else "Untitled"
        # Replace invalid characters in the title
        filename = "".join(x for x in title if x.isalnum() or x in [' ', '-', '_']) + ".txt"
        # Write the text content to a file
        with open(os.path.join(text_folder, filename), 'w', encoding='utf-8') as f:
            f.write(text_content)
        print(f"Text content saved: {url}")

        # Check for embedded files (videos, PDFs)
        for embed in soup.find_all(["iframe", "embed", {"type": "application/pdf"}]):
            src = embed.get("src")
            if src:
                # Download the file
                download_file(urljoin(url, src), folder)
                print(f"Embedded file downloaded: {src}")

        # Check for embedded videos
        for video in soup.find_all('video'):
            src = video.get("src")
            if src:
                # Download the video
                download_file(urljoin(url, src), folder)
                print(f"Video downloaded: {src}")
    else:
        print(f"Failed to download page: {url}")

# Function to download files (videos, PDFs)
def download_file(url, folder):
    # Extract filename from URL
    filename = os.path.basename(urllib.parse.urlsplit(url).path)
    # Create a folder for files if it doesn't exist
    files_folder = os.path.join(folder, "files")
    if not os.path.exists(files_folder):
        os.makedirs(files_folder)
    # Download the file
    with open(os.path.join(files_folder, filename), 'wb') as f:
        f.write(requests.get(url).content)

# URL of the website
base_url = "https://am.jpmorgan.com/us/en/asset-management/liq/"

# Fetch the homepage
homepage_url = base_url
download_page(homepage_url, "webpages")

# Parse the homepage to find links to other pages
response = requests.get(homepage_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all <a> tags with href attribute
links = soup.find_all('a', href=True)

# Iterate over each link
for link in links:
    # Get the URL of the link
    url = urljoin(base_url, link['href'])
    # Filter out URLs that don't start with "http" or "https"
    if url.startswith("http://") or url.startswith("https://"):
        # Download the page
        download_page(url, "webpages")


Text content saved: https://am.jpmorgan.com/us/en/asset-management/liq/
Embedded file downloaded: https://www.googletagmanager.com/ns.html?id=GTM-5ZSQBVT
Text content saved: https://am.jpmorgan.com/us/en/asset-management/liq/#contentStart
Embedded file downloaded: https://www.googletagmanager.com/ns.html?id=GTM-5ZSQBVT
Text content saved: https://am.jpmorgan.com/us/en/asset-management/liq/
Embedded file downloaded: https://www.googletagmanager.com/ns.html?id=GTM-5ZSQBVT
Text content saved: https://am.jpmorgan.com/us/en/asset-management/liq/funds/
Embedded file downloaded: https://www.googletagmanager.com/ns.html?id=GTM-5ZSQBVT
Text content saved: https://am.jpmorgan.com/us/en/asset-management/liq/funds/
Embedded file downloaded: https://www.googletagmanager.com/ns.html?id=GTM-5ZSQBVT
Text content saved: https://am.jpmorgan.com/us/en/asset-management/liq/products/fund-explorer
Embedded file downloaded: https://www.googletagmanager.com/ns.html?id=GTM-5ZSQBVT
Text content saved: https://a

With Cosine similarity similarity measure

In [4]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [20]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin
import urllib
from sentence_transformers import SentenceTransformer, util
import numpy as np

def find_similar_pages(target_url):
    # Function to download webpage content
    def download_page(url):
        # Make a GET request to the URL
        response = requests.get(url)
        # If the request is successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')
            # Extract text content
            text_content = soup.get_text()
            return text_content
        else:
            print(f"Failed to download page: {url}")
            return None

    # Function to check if page content is already downloaded
    def is_downloaded(url):
        # Check if the text file exists
        text_file_path = os.path.join("webpages", "text", url.replace("/", "_").replace(":", "") + ".txt")
        return os.path.exists(text_file_path)

    # Function to ensure directory structure exists
    def ensure_directory_structure():
        # Create directories if they don't exist
        directory = os.path.join("webpages", "text")
        if not os.path.exists(directory):
            os.makedirs(directory)

    # Ensure directory structure exists
    ensure_directory_structure()

    # URL of the website
    base_url = "https://am.jpmorgan.com/us/en/asset-management/liq/"

    if is_downloaded(target_url):
        # If content is already downloaded, read from file
        with open(os.path.join("webpages", "text", target_url.replace("/", "_").replace(":", "") + ".txt"), 'r', encoding='utf-8') as f:
            target_content = f.read()
    else:
        # If content is not downloaded, download and save to file
        target_content = download_page(target_url)
        if target_content:
            with open(os.path.join("webpages", "text", target_url.replace("/", "_").replace(":", "") + ".txt"), 'w', encoding='utf-8') as f:
                f.write(target_content)

    if target_content:
        # Parse the target page to find links to other pages
        response = requests.get(target_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find all <a> tags with href attribute
        links = soup.find_all('a', href=True)

        # List to store URLs and content of all pages
        all_urls = [target_url]
        all_contents = [target_content]

        # Initialize BERT model
        model = SentenceTransformer('bert-base-nli-mean-tokens')

        # Iterate over each link
        for link in links:
            # Get the URL of the link
            url = urljoin(base_url, link['href'])
            # Filter out URLs that don't start with "http" or "https"
            if url.startswith("http://") or url.startswith("https://"):
                # Check if content is already downloaded
                if is_downloaded(url):
                    # If content is already downloaded, read from file
                    with open(os.path.join("webpages", "text", url.replace("/", "_").replace(":", "") + ".txt"), 'r', encoding='utf-8') as f:
                        content = f.read()
                else:
                    # If content is not downloaded, download and save to file
                    content = download_page(url)
                    if content:
                        with open(os.path.join("webpages", "text", url.replace("/", "_").replace(":", "") + ".txt"), 'w', encoding='utf-8') as f:
                            f.write(content)
                if content:
                    # Add URL and content to lists
                    all_urls.append(url)
                    all_contents.append(content)

        # Compute embeddings for all content
        content_embeddings = model.encode(all_contents)

        # Compute similarities between the target page and all other pages
        target_index = 0  # Index of the target page
        similarities = util.pytorch_cos_sim(content_embeddings[target_index].reshape(1, -1), content_embeddings)

        # Find top 5 similar pages
        similar_indices = np.argsort(similarities[0].cpu().numpy())[::-1][1:6]  # Exclude the target page itself
        similar_urls = [all_urls[index] for index in similar_indices]

        return similar_urls
    else:
        print("Failed to find content for the target page.")
        return []

# Example usage:
target_url = "https://am.jpmorgan.com/us/en/asset-management/liq/insights/market-insights/esg-explained-7-essentials-you-need-to-know/"
similar_pages = find_similar_pages(target_url)
if similar_pages:
    print("Top 5 similar pages:")
    for i, url in enumerate(similar_pages):
        print(f"{i+1}. {url}")
else:
    print("No similar pages found.")


Failed to download page: https://twitter.com/intent/tweet?url=https://am.jpmorgan.com/us/en/asset-management/liq/insights/market-insights/esg-explained-7-essentials-you-need-to-know/&original_referer=https://am.jpmorgan.com/us/en/asset-management/liq/insights/market-insights/esg-explained-7-essentials-you-need-to-know/
Failed to download page: https://www.linkedin.com/company/jpmorganassetmanagement
Top 5 similar pages:
1. https://am.jpmorgan.com/us/en/asset-management/liq/insights/market-insights/esg-explained-7-essentials-you-need-to-know/
2. https://am.jpmorgan.com/us/en/asset-management/liq/about-us/diversity-inclusion/
3. https://am.jpmorgan.com/us/en/asset-management/liq/insights/portfolio-insights/ltcma/
4. https://am.jpmorgan.com/us/en/asset-management/liq/resources/investment-academy/
5. https://am.jpmorgan.com/us/en/asset-management/liq/insights/liquidity-insights/leveraging-the-power-of-cash-segmentation/
